In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics


In [2]:
!ls ../../

ChessBot  data-goodmove.csv


In [3]:
import random

nrow = 10000 #desired sample size
n = sum(1 for line in open('../../data-goodmove.csv')) - 1 #number of records in file (excludes header)

skiprows = np.sort(np.random.choice(np.arange(n), n - nrow, replace= False))

In [4]:
df = pd.read_csv('../../data-goodmove.csv', skiprows=skiprows)
print(df.shape)

(10000, 193)


In [5]:
df = df.replace(['None', 'P', 'N', 'B', 'R', 'Q', 'K', 'p', 'n', 'b', 'r', 'q', 'k'], [0, 1, 3, 3, 5, 9, 100, -1, -3, -3, -5, -9, -100])

In [6]:
X_dataFrame, y_dataFrame = df.iloc[:, :192], df.iloc[:, 192]

In [7]:
from sklearn.preprocessing import OneHotEncoder

X = X_dataFrame.to_numpy()
y = y_dataFrame.to_numpy().reshape(-1, 1)

In [8]:
X_train, X_test , y_train,  y_test = train_test_split(X, y, test_size=0.25)
del (X, y, X_dataFrame, y_dataFrame)

In [9]:
encY = OneHotEncoder().fit(y_train)
encY.inverse_transform([[1, 0], [0, 1]])

array([[0],
       [1]])

In [10]:
Y_train = np.array(encY.transform(y_train).toarray())
Y_train.shape

(7500, 2)

In [11]:
class f1_loss(nn.Module):
  def __init__(self, weight_class = None, weight_tp = None):
    super(f1_loss, self).__init__()
    self.weight_class = weight_class
    self.weight_tp = weight_tp


  def forward(self, Y_pred, Y_true):

    tp = torch.sum(Y_true*Y_pred, dim=0)
    
    fp = torch.sum((1-Y_true)*Y_pred, dim=0)
    fn = torch.sum(Y_true*(1-Y_pred), dim=0)

    if self.weight_tp is not None:
      tp = tp * torch.Tensor(self.weight_tp)

    p = tp / (tp + fp + 1e-10)
    r = tp / (tp + fn + 1e-10)

    f1 = 2*p*r / (p+r+1e-10)
    f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)
    if self.weight_class is not None:
      # NOTE 
      # f1 = f1 * torch.Tensor(self.weight).cuda()
      f1 = f1 * torch.Tensor(self.weight_class)


    return - torch.mean(f1)

In [12]:
class BuildDataSet(Dataset):
    def __init__ (self, X, Y = np.empty((0, 2))):
        super().__init__()
        self.size = X.shape[0]
        self.data = torch.Tensor(X)
        if len(Y) == 0:
            self.is_testSet = True
        else:
            self.is_testSet = False
            self.label = torch.Tensor(Y)

    
    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        if self.is_testSet:
            return self.data[idx]
        return self.data[idx], self.label[idx]

In [13]:
class DeepLearningModel(nn.Module):
    def __init__(self, in_features):
        super(DeepLearningModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_features = in_features, out_features = 50),
            nn.ReLU(),
            nn.Linear(50, 2),
            # nn.ReLU(),
            nn.Softmax(1)
        )
    
    def forward(self, x):
        out = self.fc(x)
        return out

In [14]:
def Train(num_epochs, model, dataset, loss_func, batch_size = 32):

    loaders = DataLoader(dataset, batch_size)
    total_step = len(loaders)

    model.train()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(num_epochs):
        for i, (x, y) in enumerate(loaders):
            # x, y = x.type(torch.float).to('cuda'), y.type(torch.float).to('cuda')
            out = model(x)
            loss = loss_func(out, y)

            optimizer.zero_grad()

            loss.backward()
            optimizer.step()

            if (i+1) % 20 == 0:
                
                Y_pred = Predict(DLmodel, torch.Tensor(X_train))
                y_pred = encY.inverse_transform(Y_pred).reshape(-1,)
                y_label = y_train.reshape(-1, )
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, {}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), metrics.f1_score(y_label, y_pred, average='macro')))


In [15]:
def Predict(model, X_loaders):
    Y_pred = model(X_loaders).detach().numpy()
    Y_pred = np.where(Y_pred < 0.5, 0, 1)
    return Y_pred

In [16]:
train_dataset =  BuildDataSet(X_train, Y_train)

In [17]:
DLmodel = DeepLearningModel(192)

In [18]:
loss_func = f1_loss([197, 7303], [1, 10])

In [19]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    DLmodel = DLmodel.cuda()
    train_dataset.to('cuda')

Train(400, DLmodel, train_dataset, loss_func, 64)

Epoch [1/400], Step [20/118], Loss: -349.0036, 0.452029246925473
Epoch [1/400], Step [40/118], Loss: -1183.7557, 0.42635213016656526
Epoch [1/400], Step [60/118], Loss: -1956.4268, 0.27136450918626037
Epoch [1/400], Step [80/118], Loss: -1002.4420, 0.1769972042201567
Epoch [1/400], Step [100/118], Loss: -34.8912, 0.20156574103384436
Epoch [2/400], Step [20/118], Loss: -843.0798, 0.37139443134177136
Epoch [2/400], Step [40/118], Loss: -2408.0437, 0.409313912125926
Epoch [2/400], Step [60/118], Loss: -2060.2954, 0.44709520872315756
Epoch [2/400], Step [80/118], Loss: -318.5963, 0.4893955120792936
Epoch [2/400], Step [100/118], Loss: -87.2278, 0.49700536748098817
Epoch [3/400], Step [20/118], Loss: -1679.8433, 0.46540968212794837
Epoch [3/400], Step [40/118], Loss: -2314.4004, 0.38628478808812244
Epoch [3/400], Step [60/118], Loss: -2216.9214, 0.4054454879451004
Epoch [3/400], Step [80/118], Loss: -164.6610, 0.4492060506730433
Epoch [3/400], Step [100/118], Loss: -91.0757, 0.5046525214037

In [20]:

Y_pred = Predict(DLmodel, torch.Tensor(X_train))
y_pred = encY.inverse_transform(Y_pred).reshape(-1,)
y_label = y_train.reshape(-1, )

print(confusion_matrix(y_pred, y_label))
print(metrics.f1_score(y_label, y_pred, average='macro'))
print(metrics.classification_report(y_label, y_pred))


[[7215   90]
 [  58  137]]
0.8195684076310535
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7273
           1       0.70      0.60      0.65       227

    accuracy                           0.98      7500
   macro avg       0.85      0.80      0.82      7500
weighted avg       0.98      0.98      0.98      7500



In [21]:
Y_pred = Predict(DLmodel, torch.Tensor(X_test))
y_pred = encY.inverse_transform(Y_pred).reshape(-1,)
y_label = y_test.reshape(-1, )

print(confusion_matrix(y_label, y_pred))
print(metrics.f1_score(y_label, y_pred, average='macro'))
print(metrics.classification_report(y_label, y_pred))

[[2378   77]
 [  45    0]]
0.4874948749487495
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2455
           1       0.00      0.00      0.00        45

    accuracy                           0.95      2500
   macro avg       0.49      0.48      0.49      2500
weighted avg       0.96      0.95      0.96      2500

